In [1]:
import pandas as pd
import mlflow
import mlflow.xgboost
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from mlflow.models import infer_signature
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

# 데이터 로딩 및 전처리
data = pd.read_csv('../data/churn.csv')

# Tracking Server 설정
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000") # 환경 변수를 설정했지만, 확실하게 설정

In [2]:
X = data.drop(['Exited', 'RowNumber', 'CustomerId', 'Surname'], axis=1)
y = data['Exited']

In [3]:
categorical_features = ['Geography', 'Gender']
numeric_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']

In [4]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [5]:
X_processed = preprocessor.fit_transform(X)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

In [7]:
# 하이퍼파라미터 탐색 공간 정의
space = {
    'max_depth': hp.choice('max_depth', range(3, 10)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'n_estimators': hp.choice('n_estimators', range(50, 200)),
    'gamma': hp.uniform('gamma', 0, 5)
}

In [8]:
# objective 함수 정의
def objective(params):
    with mlflow.start_run(nested=True):
        model = xgb.XGBClassifier(
            use_label_encoder=False,
            eval_metric='logloss',
            **params
        )
        model.fit(X_train, y_train)
        
        # 예측값과 확률 분리
        preds = model.predict(X_test)
        probs = model.predict_proba(X_test)[:, 1]

        acc = accuracy_score(y_test, preds)
        precision = precision_score(y_test, preds)
        recall = recall_score(y_test, preds)
        f1 = f1_score(y_test, preds)
        roc_auc = roc_auc_score(y_test, probs)

        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': acc,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'roc_auc': roc_auc
        })

        signature = infer_signature(X_train, probs)

        return {
            'loss': -roc_auc,
            'status': STATUS_OK,
            'model': model,
            'signature': signature
        }

In [9]:
# 실험 설정 및 fmin 수행
mlflow.set_experiment("practice5")

with mlflow.start_run(run_name="XGBoost") as run:
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=20,
        trials=trials
    )

    # best run 결과 추출
    best_result = sorted(trials.results, key=lambda x: x["loss"])[0]
    best_model = best_result["model"]
    signature = best_result["signature"]

    mlflow.log_params(best)
    mlflow.log_metric("best_roc_auc", -best_result["loss"])

    # 최종 best 모델 저장 (루트 run에)
    mlflow.xgboost.log_model(best_model, "best_xgboost_model", signature=signature)

    print(f"Best parameters: {best}")
    print(f"Best ROC-AUC: {-best_result['loss']:.4f}")

2025/10/10 23:56:40 INFO mlflow.tracking.fluent: Experiment with name 'practice5' does not exist. Creating a new experiment.


  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

c:\Users\tmdtn\venvs\mlflow-env\Lib\site-packages\xgboost\training.py:183: UserWarning: [23:56:42] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



🏃 View run legendary-lamb-99 at: http://127.0.0.1:5000/#/experiments/1/runs/75897464a9494388bf272ab2bd425079

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1

  5%|▌         | 1/20 [00:01<00:26,  1.37s/trial, best loss: -0.871954125636726]

c:\Users\tmdtn\venvs\mlflow-env\Lib\site-packages\xgboost\training.py:183: UserWarning: [23:56:44] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



...

🏃 View run adventurous-loon-943 at: http://127.0.0.1:5000/#/experiments/1/runs/ab72710ea1d64f46ad6814686a2cd68e

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                      

 95%|█████████▌| 19/20 [00:18<00:00,  1.09trial/s, best loss: -0.8753085657373672]

c:\Users\tmdtn\venvs\mlflow-env\Lib\site-packages\xgboost\training.py:183: UserWarning: [23:57:01] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



🏃 View run legendary-squid-80 at: http://127.0.0.1:5000/#/experiments/1/runs/5001c0ce02ec4c58b5d9c208c216b8b6

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                      

100%|██████████| 20/20 [00:19<00:00,  1.02trial/s, best loss: -0.8753085657373672]


c:\Users\tmdtn\venvs\mlflow-env\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [23:57:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)


Best parameters: {'gamma': 2.175485630996258, 'learning_rate': 0.13314541619495712, 'max_depth': 2, 'n_estimators': 18}
Best ROC-AUC: 0.8753
🏃 View run XGBoost at: http://127.0.0.1:5000/#/experiments/1/runs/34850d987173402bbb32cae4d9f89456
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
